<a href="https://colab.research.google.com/github/XiaotianF/NLP2022-23/blob/main/wmt18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate
!pip uninstall accelerate
!pip install accelerate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, EncoderDecoderModel
import evaluate
import numpy as np
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq


train_dataset = load_dataset("wmt18", "zh-en", split='train[:2%]')
val_dataset = load_dataset("wmt18", "zh-en", split='validation[:10%]')

Extracting data files:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/25160346 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3981 [00:00<?, ? examples/s]

In [3]:
en_texts = []
zh_texts =[]


for item in train_dataset['translation']:
    en_texts.append(item['en'])
    zh_texts.append(item['zh'])

val_en_texts = []
val_zh_texts = []

for item in val_dataset["translation"]:
    val_en_texts.append(item['en'])
    val_zh_texts.append(item['zh'])

In [4]:
ds_train = pd.DataFrame({'en':en_texts, 'zh':zh_texts})
ds_val = pd.DataFrame({'en':val_en_texts, 'zh':val_zh_texts})

print(ds_val['zh'].head())

0                                    28岁厨师被发现死于旧金山一家商场
1                   近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。
2                  但受害人的哥哥表示想不出有谁会想要加害于他，并称“一切终于好起来了。”
3    旧金山验尸官办公室表示，周三早上于西田购物中心发现的尸体确认为28岁旧金山居民 Frank ...
4                         旧金山警察局称该起死亡案件被裁定为他杀，并正在进行调查。
Name: zh, dtype: object


In [5]:
model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=512)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_checkpoint, model_checkpoint)

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.3.crossattention.self.key.weight', 'bert.encoder.layer.4.crossattention.output.dense.bias', 'bert.encoder.layer.11.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.11.crossattention.self.value.bias', 'bert.encoder.layer.10.crossattention.output.dense.bias', 'bert.encoder.layer.10.crossattention.self.query.bias', 'bert.encoder.layer.10.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.self.query.weight', 'bert.encoder.layer.8.crossattention.output.dense.weight', 'bert.encoder.layer.5.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.7.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.self.value.bias', 'bert.encoder.layer.2.crossattention.self.key.bias',

In [6]:
encoder_max_length = 512
decoder_max_length = 128

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["zh"], padding="max_length", truncation=True, max_length=encoder_max_length, return_tensors="pt")
    outputs = tokenizer(batch["en"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch


In [7]:
ds_train.to_csv("wmt11_train.csv",index=False)
ds_val.to_csv("wmt11_val.csv", index=False)


ds_train = load_dataset('csv', data_files="wmt11_train.csv")
ds_val = load_dataset('csv', data_files="wmt11_val.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
# ds_train = ds_train["train"][:320]
# ds_val = ds_val["validation"][:80]

TypeError: ignored

In [8]:
tk_train = ds_train.map(
    process_data_to_model_inputs,
    batched=True,
    remove_columns=["en", "zh"]
)

tk_val = ds_val.map(
    process_data_to_model_inputs,
    batched=True,
    remove_columns=["en", "zh"]
)

Map:   0%|          | 0/503207 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    lables = np.where(labels != -100, lables, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predications=decoded_preds, references=decoded_labels)
    result = {"blue": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Training

In [10]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size
model.config.max_length = 512
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [25]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy = "epoch",
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 3,
    push_to_hub = False,
    report_to = None,
    output_dir = "./"
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, return_tensors="pt")

In [26]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    compute_metrics=metric,
    train_dataset=tk_train['train'],
    eval_dataset=tk_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

OutOfMemoryError: ignored

## Evaluation

In [ ]:
test_dataset = load_dataset("wmt18", "zh-en", split='test[:10%]')

test_en_texts =[]
test_zh_texts = []

for item in train_dataset['translation']:
    test_en_texts.append(item['en'])
    test_zh_texts.append(item['zh'])

ds_test = pd.DataFrame({'en':test_en_texts, 'zh':test_zh_texts})

In [ ]:
def translation(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["zh"],  padding="max_length", truncation=True, max_length=encoder_max_length, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=128, decoder_start_token_id=model.config.decoder.pad_token_id)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_en"] = output_str

    return batch

In [ ]:
batch_size = 8
outcomes = ds_test.map(translation, batched=True, batch_size=batch_size, remove_columns=["zh"])

In [ ]:
outcomes = pd.DataFrame(outcomes)
outcomes.head()

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from datasets import load_metric

#load bleu metric
bleu_metric = load_metric["bleu"]

reference = outcomes["en"]
generated_translations = outcomes["pre_en"]

references = [[ref] for ref in regerences]
generated_translations = [gen for gen in generated_translations]

bleu_scores = bleu_metric.compute(predictions=generated_translations, references=references)["bleu"]

average_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average Bleu score:", average_bleu)

In [ ]:
# source_lang = "zh"
# target_lang = "en"

# prefix = "translate Chinese to English:"

# def preprocess_function(examples):
#     inputs = [prefix + example[source_lang] for example in examples['zh']]
#     targets = [example[target_lang] for example in examples['en']]

#     model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
#     return model_inputs

In [ ]:
# import pandas as pd

# train_en_texts = dt_rain['en']
# train_zh_texts = ds_train['zh']

# val_en_texts = val_dataset['translation']['en']
# val_zh_texts = val_dataset['translation']['zh']
